In [1]:
!pip install regex # A sligthly better version of RE
!pip install html5lib # For Pandas read_html used to parse mapping wikitables 
!pip install pandas # A great library for data wrangling (and analysis) 

In [2]:
import pandas as pd 
import numpy as np
import regex
import os 

# Metadata and mappings
The template mapping can be found here:

Phabricator task [T139724](https://phabricator.wikimedia.org/T139724)

The original metadata is a Google Spreadsheet located here:

https://docs.google.com/spreadsheets/d/1s6l0Ms_14_b3wZhPHGgbCYFCs0nrKlD9E-Uxk_r3Mnc/edit?ts=57737736#gid=0

The doc below is downloaded locally 2016-07-19 16:55

In [3]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text
    
it_converters = {"Folder":strip, "Nome foto":strip, "Anno":strip, "Luogo":strip, "Nome monumento":strip, "Descrizione":strip, "Nome autore":strip}
metadata_it = pd.read_excel("./data/COH_GAR_metadata.xlsx",sheetname="Italian", skiprows=[1], converters=it_converters) # empty first row
metadata_it.columns = ["Folder","Nome_foto","Anno","Luogo","Nome_monumento","Descrizione","Nome_autore"]

en_converters = {"Title":strip, "Year":strip, "Place": strip, "Subject":strip, "Description":strip, "Author":strip, "Commons_category":strip}
metadata_en = pd.read_excel("./data/COH_GAR_metadata.xlsx",sheetname="English", converters=en_converters)
metadata_en.columns = ["Title","Year","Place","Subject","Description","Author","Commons_category","Comment"]


In [4]:
metadata_it.head()

,Folder,Nome_foto,Anno,Luogo,Nome_monumento,Descrizione,Nome_autore
0,1,Aleppo_Cittadella_01,2006,Aleppo,Cittadella,Porta di ingresso,GAR_Ilario Di Nardo
1,1,Amrit_Sepolcro_01,2006,Amrit,Meghazil Sepolcro,NaN,GAR_Ilario Di Nardo
2,1,Bosra_Teatro Romano_01,2006,Bosra,Teatro Romano,"Bosra è un'antica città nel sud della Siria, p...",GAR_Ilario Di Nardo
3,1,Damasco_Tempio di Giove_01,2006,Damasco,Tempio di Giove,Rovine del tempio di Giove all'ingresso del Su...,GAR_Ilario Di Nardo
4,1,Dura Europos_Panorama_01,2006,Dura Europos,Panorama,NaN,GAR_Ilario Di Nardo


In [5]:
metadata_en.head()

,Title,Year,Place,Subject,Description,Author,Commons_category,Comment
0,Aleppo_Cittadella_01,2006,Aleppo,Citadel,NaN,GAR_Ilario Di Nardo,Exterior views of Aleppo citadel,NaN
1,Amrit_Sepolcro_01,2006,Amrit,Meghazil tomb,NaN,GAR_Ilario Di Nardo,Amrit,NaN
2,Bosra_Teatro Romano_01,2006,Bosra,Roman theater,NaN,GAR_Ilario Di Nardo,Ancient Roman theatre (Bosra),NaN
3,Damasco_Tempio di Giove_01,2006,Damascus,Ruins of Jupiter Temple at the entrance of Suq...,NaN,GAR_Ilario Di Nardo,Temple of Jupiter (Damascus),NaN
4,Dura Europos_Panorama_01,2006,Dura Europos,General view,NaN,GAR_Ilario Di Nardo,Dura Europos,NaN


In [6]:
merged = pd.concat([metadata_it,metadata_en], axis=1) 
merged.head()

,Folder,Nome_foto,Anno,Luogo,Nome_monumento,Descrizione,Nome_autore,Title,Year,Place,Subject,Description,Author,Commons_category,Comment
0,1,Aleppo_Cittadella_01,2006,Aleppo,Cittadella,Porta di ingresso,GAR_Ilario Di Nardo,Aleppo_Cittadella_01,2006,Aleppo,Citadel,NaN,GAR_Ilario Di Nardo,Exterior views of Aleppo citadel,NaN
1,1,Amrit_Sepolcro_01,2006,Amrit,Meghazil Sepolcro,NaN,GAR_Ilario Di Nardo,Amrit_Sepolcro_01,2006,Amrit,Meghazil tomb,NaN,GAR_Ilario Di Nardo,Amrit,NaN
2,1,Bosra_Teatro Romano_01,2006,Bosra,Teatro Romano,"Bosra è un'antica città nel sud della Siria, p...",GAR_Ilario Di Nardo,Bosra_Teatro Romano_01,2006,Bosra,Roman theater,NaN,GAR_Ilario Di Nardo,Ancient Roman theatre (Bosra),NaN
3,1,Damasco_Tempio di Giove_01,2006,Damasco,Tempio di Giove,Rovine del tempio di Giove all'ingresso del Su...,GAR_Ilario Di Nardo,Damasco_Tempio di Giove_01,2006,Damascus,Ruins of Jupiter Temple at the entrance of Suq...,NaN,GAR_Ilario Di Nardo,Temple of Jupiter (Damascus),NaN
4,1,Dura Europos_Panorama_01,2006,Dura Europos,Panorama,NaN,GAR_Ilario Di Nardo,Dura Europos_Panorama_01,2006,Dura Europos,General view,NaN,GAR_Ilario Di Nardo,Dura Europos,NaN


### Check that every row in the English metadata sheet contains (at least) one Commons category, no rows should be displayed below

In [7]:
merged[pd.isnull(merged.Commons_category)]

,Folder,Nome_foto,Anno,Luogo,Nome_monumento,Descrizione,Nome_autore,Title,Year,Place,Subject,Description,Author,Commons_category,Comment


### Check that all descriptions are removed, no rows shoud be displayed below

In [8]:
merged[pd.notnull(merged.Description)]

,Folder,Nome_foto,Anno,Luogo,Nome_monumento,Descrizione,Nome_autore,Title,Year,Place,Subject,Description,Author,Commons_category,Comment


### Check that all filenames are unique

In [9]:
titles = merged.Title.copy() 

In [10]:
sorted(titles[titles.str.contains("Aleppo_Cittadella")])

['Aleppo_Cittadella_01',
 'Aleppo_Cittadella_01',
 'Aleppo_Cittadella_02',
 'Aleppo_Cittadella_03',
 'Aleppo_Cittadella_04',
 'Aleppo_Cittadella_05',
 'Aleppo_Cittadella_07',
 'Aleppo_Cittadella_08',
 'Aleppo_Cittadella_09',
 'Aleppo_Cittadella_10',
 'Aleppo_Cittadella_11',
 'Aleppo_Cittadella_12',
 'Aleppo_Cittadella_13',
 'Aleppo_Cittadella_14',
 'Aleppo_Cittadella_15',
 'Aleppo_Cittadella_16',
 'Aleppo_Cittadella_17',
 'Aleppo_Cittadella_18',
 'Aleppo_Cittadella_19',
 'Aleppo_Cittadella_20',
 'Aleppo_Cittadella_21',
 'Aleppo_Cittadella_22',
 'Aleppo_Cittadella_23',
 'Aleppo_Cittadella_24',
 'Aleppo_Cittadella_25',
 'Aleppo_Cittadella_26',
 'Aleppo_Cittadella_27',
 'Aleppo_Cittadella_28',
 'Aleppo_Cittadella_29',
 'Aleppo_Cittadella_30',
 'Aleppo_Cittadella_31',
 'Aleppo_Cittadella_32',
 'Aleppo_Cittadella_33',
 'Aleppo_Cittadella_34']

In [10]:
sorted_titles = titles.sort_values

# Keyword mappings

Manual mappings of places, people and keywords for categories. They are published as wikitables and can be found here:

https://commons.wikimedia.org/w/index.php?title=Special%3APrefixIndex&prefix=Gruppo+Archeologico+Romano%2FBatch+upload%2F&namespace=4

## Places (Luogo)

We have two different mappings of places, one general based on the column "Place (Luogo)" in the original metadata file and one more specific which is a combination of the columns "Place (Luogo)" and "Subject (Nome_monumento)". 

The mapping can be found here:

https://commons.wikimedia.org/wiki/Commons:Gruppo_Archeologico_Romano/Batch_upload/places

In [11]:
place_mappings_url = "https://commons.wikimedia.org/wiki/Commons:Gruppo_Archeologico_Romano/Batch_upload/places"
place_mappings = pd.read_html(place_mappings_url, attrs = {"class":"wikitable"}, header=0)
place_mappings_general = place_mappings[0]
# Strip away potential surrounding whitespace
place_mappings_general["Luogo"] = place_mappings_general.Luogo.str.strip() 
place_mappings_general["wikidata"] = place_mappings_general.wikidata.str.strip()
place_mappings_general["category"] = place_mappings_general.category.str.strip() 
place_mappings_general["category"] = place_mappings_general.category.str.replace("_", " ") 

place_mappings_general = place_mappings_general.set_index("Luogo")

place_mappings_specific = place_mappings[1]
# Strip away potential surrounding whitespace
place_mappings_specific["Luogo"] = place_mappings_specific.Luogo.str.strip()
place_mappings_specific["Nome_monumento"] = place_mappings_specific.Nome_monumento.str.strip()

place_mappings_specific["category"] = place_mappings_specific.category.str.strip()
place_mappings_specific["category"] = place_mappings_specific.category.str.replace("_", " ")
place_mappings_specific["wikidata"] = place_mappings_specific.wikidata.str.strip()


place_mappings_specific["Specific_place"] = place_mappings_specific.Luogo + " " + place_mappings_specific.Nome_monumento
place_mappings_specific = place_mappings_specific[["Specific_place" ,"Luogo","Nome_monumento","category","wikidata"]]
place_mappings_specific = place_mappings_specific.set_index("Specific_place")

Check that all '-' are replaced with empty strings

In [12]:
print(place_mappings_specific["wikidata"].head(10))

Specific_place
Aleppo Cittadella                            d:Q206231
Palmira Tempio di Baal                       d:Q797481
Aleppo Suq                                           -
Qasr al-Hayr al-Sharqi Palazzo Omayyade     d:Q1149473
Yabroud Necropoli di Yabroud                         -
Sergiopolis Cattedrale bizantina                     -
Halabiyah Fortezza                                   -
Apamea Tra i bambini del villaggio arabo             -
Mari Palazzo di Zimri-Lin                   d:Q1887409
Palmyra Interno dell'Ipogeo di Artaban               -
Name: wikidata, dtype: object


In [13]:
place_mappings_specific["wikidata"].head(10)

Specific_place
Aleppo Cittadella                            d:Q206231
Palmira Tempio di Baal                       d:Q797481
Aleppo Suq                                           -
Qasr al-Hayr al-Sharqi Palazzo Omayyade     d:Q1149473
Yabroud Necropoli di Yabroud                         -
Sergiopolis Cattedrale bizantina                     -
Halabiyah Fortezza                                   -
Apamea Tra i bambini del villaggio arabo             -
Mari Palazzo di Zimri-Lin                   d:Q1887409
Palmyra Interno dell'Ipogeo di Artaban               -
Name: wikidata, dtype: object

In [14]:
place_mappings_specific.loc["Serjilla Andron"]["wikidata"]

'-'

In [15]:
pd.isnull(place_mappings_specific.loc["Serjilla Andron"]["wikidata"])

False

In [16]:
place_mappings_specific.loc["Serjilla Andron"]["wikidata"] == "-"

True

In [17]:
pd.isnull(place_mappings_specific.wikidata)

Specific_place
Aleppo Cittadella                                                                 False
Palmira Tempio di Baal                                                            False
Aleppo Suq                                                                        False
Qasr al-Hayr al-Sharqi Palazzo Omayyade                                           False
Yabroud Necropoli di Yabroud                                                      False
Sergiopolis Cattedrale bizantina                                                  False
Halabiyah Fortezza                                                                False
Apamea Tra i bambini del villaggio arabo                                          False
Mari Palazzo di Zimri-Lin                                                         False
Palmyra Interno dell'Ipogeo di Artaban                                            False
Hama Centro Storico di Hama - particolare                                         False
Burjikeh rovine b

In [18]:
place_mappings_general.head(3)

,frequency,category,wikidata
Luogo,,,
Aleppo,75,Category:Aleppo,d:Q41183
Palmira,44,Category:Palmyra,d:Q5747
Palmyra,40,Category:Palmyra,d:Q5747


In [19]:
place_mappings_specific.head(3)

,Luogo,Nome_monumento,category,wikidata
Specific_place,,,,
Aleppo Cittadella,Aleppo,Cittadella,Category:Aleppo citadel,d:Q206231
Palmira Tempio di Baal,Palmira,Tempio di Baal,Category:Temple of Baal in Palmyra,d:Q797481
Aleppo Suq,Aleppo,Suq,Category:Suqs and Khans in Aleppo,-


# Population of the Photograph template

## Template mapping

 The master template mapping lives as [a task on Phabricator](https://phabricator.wikimedia.org/T139724)

## Collect original filenames

In [20]:
sub_dirs = []
original_filenames = []

root_dir = "/Users/mos/GAR pictures for WMSE/"
for root, dirs, files in os.walk("/Users/mos/GAR pictures for WMSE/"):
    sub_dirs.extend(dirs)
for dir in sub_dirs:
    for root, dirs, files in os.walk(root_dir + dir):
        for file in files:
            if file.endswith(".JPG") or file.endswith(".jpg"):
                original_filenames.append(file)
print("Subdirs: {}\nlen(original_filenames): {}\nFiles in metadata file: 426".format(len(sub_dirs), len(original_filenames)))

Subdirs: 0
len(original_filenames): 0
Files in metadata file: 426


## Create wikitext for image pages
Available as .py script on [my github](https://github.com/mattiasostmar/GAR_Syria_2016-06/blob/master/create_metatdata_textfiles.py)

In [25]:
# remove possible diuplicate files with other extension names
!rm -rf ./photograph_template_texts/*

total_images = 0
OK_images = 0
uncategorized_images = 0
faulty_images = 0

filenames_file = open("./filenames_mapping.csv","w")
filenames_file.write("Folder|Original|Commons\n")

for row_no, row in merged.iterrows():
    # Filename: <Nome_foto[:-3]>_GAR_<Nome_foto[-3:]>.<ext>
    outpath = "./photograph_template_texts/"
    nome_foto = row["Nome_foto"].replace(" ", "_")
    nome_foto_0, dummy, nome_foto_1 = nome_foto.rpartition("_")
    fname = str(nome_foto_0) + "_-_" + "GAR" + "_-_" + str(row["Folder"]) + "-" + str(nome_foto_1) # + ".JPG" Hack, extension ought to be dynamic
    print("{}".format(fname))

    filenames_file.write("{}|{}|{}\n".format(row["Folder"],row["Nome_foto"],fname))
    
    total_images += 1
    
    template_parts = []
    
    header = "{{Photograph"
    template_parts.append(header)
    
    if not pd.isnull(row["Author"]): 
        photographer = "|photographer = " + row["Author"][4:]
    else:
        print("Warning! Empty Author column in row no {} photo {}".format(row_no, row["Nome_foto"]))
        photographer = "|photographer = "
        faulty_images += 1
    template_parts.append(photographer)
    
    title_it = "{{it|'''" + regex.sub("_"," ",nome_foto_0) + "'''}}"
    #title_en = "{{en|" + regex.sub("_"," ",row["Title"][:-3]) + "}}"
    
    title = "|title = " + title_it #+ "\n" + title_en
    template_parts.append(title)
    
    if pd.notnull(row["Description"]):
        description_en = "{{en|" + row["Description"] + "}}" # <Description> is empty though, not translated
    else:
        description_en = "{{en|" + str(row["Subject"]) + ", " + str(row["Place"]) + " in " + str(row["Year"]) + "}}"
    
    if pd.notnull(row["Descrizione"]):
        description_it = "{{it|" + row["Descrizione"] + "}}"
    else:
        description_it = "{{it|" + str(row["Nome_monumento"]) + ", " + str(row["Luogo"]) + ", " + str(row["Anno"]) + "}}"
    
    description = "|description = " + description_it + "\n" + description_en
    template_parts.append(description)
    
    depicted_people = "|depicted people ="
    template_parts.append(depicted_people)
    # Workoaround that we don't have actual specific places in mapping table
    spec_place = row["Luogo"] + " " + row["Nome_monumento"]
    
    if not place_mappings_specific.loc[spec_place]["wikidata"] == "-" and pd.notnull(place_mappings_specific.loc[spec_place]["wikidata"]): 
        depicted_place = "|depicted place = {{city|" + \
        place_mappings_specific.loc[spec_place]["wikidata"][2:] + "}}" #[2:] since "d:" begins wikidata string
        #print(depicted_place)
    elif not place_mappings_general.loc[row["Luogo"]]["wikidata"] == "-" or pd.isnull(place_mappings_general.loc[row["Luogo"]]["wikidata"]):
        depicted_place = "|depicted place = {{city|" + \
        place_mappings_general.loc[row["Luogo"]]["wikidata"][2:] + "}}" #[2:] since "d:" begins wikidata string
        #print(depicted_place)
    else:
        depicted_place = "|depicted place = " + row["Nome_monumento"] + ", " + row["Luogo"]
        #print(depicted_place)
    template_parts.append(depicted_place)
    
    if pd.notnull(row["Year"]):
        date = "|date = " + str(row["Year"])
    else:
        date = "|date = "
    template_parts.append(date)
        
    medium = "|medium =" 
    template_parts.append(medium)
    
    dimensions = "|dimensions ="
    template_parts.append(dimensions)
    
    institution = "|institution = {{Institution:Gruppo Archeologico Romano}}"
    template_parts.append(institution)
    
    department = "|department ="
    template_parts.append(department)
    
    references = "|references ="
    template_parts.append(references)
    
    object_history = "|object history ="
    template_parts.append(object_history)
    
    exhibition_history = "|exhibition history ="
    template_parts.append(exhibition_history)
    
    credit_line = "|credit line ="
    template_parts.append(credit_line)
    
    inscriptions = "|inscriptions ="
    template_parts.append(inscriptions)
    
    notes = "|notes ="
    template_parts.append(notes)
    
    accession_number = "|accession number ="
    template_parts.append(accession_number)
    
    source = "|source = ''" + str(row["Folder"]) + "/" + str(nome_foto) + "''\n{{Gruppo Archeologico Romano cooperation project|COH}}"
    template_parts.append(source)
    
    if pd.notnull(row["Author"]):
        permission = "|permission = {{CC-BY-SA-4.0|" + row["Author"][4:] + " / GAR}}\n{{PermissionOTRS|id=2016042410005958}}"
    else:
        permission = "|permission = {{CC-BY-SA-4.0|Gruppo Archeologico Romano}}\n{{PermissionOTRS|id=2016042410005958}}"
    template_parts.append(permission)
    
    other_versions = "|other_versions ="
    template_parts.append(other_versions)
    
    template_parts.append("}}")
    
    
    categories_list = []
    # [[Category:<Category from Specific Place> AND/OR <Category from Luogo (place)>]] 
    specific_place_category = None
    general_place_category = None
    maintanence_category = None
    batchupload_category = "[[Category:Images_from_GAR_2016-06]]"
    translation_needed_category = "[[Category:Images_from_GAR_needing_English_description]]"
    
    if place_mappings_specific.loc[spec_place]["category"] != "-" and pd.notnull(place_mappings_specific.loc[spec_place]["category"]): 
        
        specific_place_category = "[[" + place_mappings_specific.loc[spec_place]["category"] + "]]"
        print("specific_place_category{}".format(specific_place_category)) 
     
    elif place_mappings_general.loc[row["Luogo"]]["category"] != "-" and pd.notnull(place_mappings_general.loc[row["Luogo"]]["category"]):
        general_place_category = "[[" + place_mappings_general.loc[row["Luogo"]]["category"] + "]]"
        print("general_place_category: {}".format(general_place_category))
    
    # [[Category:Images_from_GAR_Syria_2016-06]]
    else:
        maintanence_category = "[[Category:Images_from_GAR_without_categories]]"
        print("maintanence_category: {}".format(maintanence_category))
    
    # manage content categories
    if specific_place_category :
        categories_list.append(specific_place_category)
        
    elif general_place_category and not specific_place_category:
        categories_list.append(general_place_category) 
    
    Commons_category = "[[Category:" + row["Commons_category"] + "]]"
    if regex.search(r" \+ ",Commons_category):
        cats = regex.split(r" \+ ",Commons_category)
        print("{} is really {}".format(row.Commons_category, cats))
        
        for cat_no, cat in enumerate(cats):
            if cat_no == 0:
                Commons_category = cat + "]]" 
            else:
                Commons_category = "[[Category:" + cat
            
            if Commons_category != specific_place_category and Commons_category != general_place_category:
                categories_list.append(Commons_category)
                print("Commons_category: {}".format(Commons_category))
            else:
                pass
    else:
        pass
    
    if Commons_category != specific_place_category and Commons_category != general_place_category:
            categories_list.append(Commons_category)
            print("Commons_category: {}".format(Commons_category))
                
    if categories_list == None:
        print("categories_list is None")
        categories_list.append(maintanence_category)
        faulty_images += 1

    categories_list.append(batchupload_category)
    categories_list.append(translation_needed_category)
    
    if len(categories_list) >0:
        OK_images += 1
    print(categories_list)
    print()
    
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    outfile = open(outpath + fname + ".info", "w")
    outfile.write("\n".join(template_parts) + "\n" + "\n".join(categories_list))

filename_file.close()    
print("Stats: \nTotal images {}\nOK images {}\nUncategorized images {}\nImages missing author {}".format(total_images, OK_images - faulty_images, uncategorized_images, faulty_images ))

Aleppo_Cittadella_-_GAR_-_1-01
specific_place_category[[Category:Aleppo citadel]]
Commons_category: [[Category:Exterior views of Aleppo citadel]]
['[[Category:Aleppo citadel]]', '[[Category:Exterior views of Aleppo citadel]]', '[[Category:Images_from_GAR_2016-06]]', '[[Category:Images_from_GAR_needing_English_description]]']

Amrit_Sepolcro_-_GAR_-_1-01
general_place_category: [[Category:Amrit]]
['[[Category:Amrit]]', '[[Category:Images_from_GAR_2016-06]]', '[[Category:Images_from_GAR_needing_English_description]]']

Bosra_Teatro_Romano_-_GAR_-_1-01
general_place_category: [[Category:Bosra]]
Commons_category: [[Category:Ancient Roman theatre (Bosra)]]
['[[Category:Bosra]]', '[[Category:Ancient Roman theatre (Bosra)]]', '[[Category:Images_from_GAR_2016-06]]', '[[Category:Images_from_GAR_needing_English_description]]']

Damasco_Tempio_di_Giove_-_GAR_-_1-01
general_place_category: [[Category:Damascus]]
Commons_category: [[Category:Temple of Jupiter (Damascus)]]
['[[Category:Damascus]]', '

NameError: name 'filename_file' is not defined

In [142]:
for row_no, row in merged.iterrows():
    print(pd.notnull(place_mappings_specific.loc[row["Luogo"] + " " + row["Nome_monumento"]]["category"]))

True
False
False
False
False
False
False
True
False
False
False
False
True
True
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
True
True
True
False
True
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True

In [21]:
!ls -la ./photograph_template_texts/ | head -n10

total 3304
drwxr-xr-x@ 416 mos  staff  14144 12 Jul 14:47 .
drwxr-xr-x@  17 mos  staff    578 18 Jul 10:40 ..
-rw-r--r--@   1 mos  staff    762 18 Jul 10:41 APAMEA Acropoli con villaggio arabo _GAR_014
-rw-r--r--@   1 mos  staff    794 18 Jul 10:41 APAMEA Case e botteghe lungo la Via colonnata _GAR_052
-rw-r--r--@   1 mos  staff    794 18 Jul 10:41 APAMEA Case e botteghe lungo la Via colonnata _GAR_053
-rw-r--r--@   1 mos  staff    898 18 Jul 10:41 APAMEA Il Tetraconco _GAR_009
-rw-r--r--@   1 mos  staff    742 18 Jul 10:41 APAMEA Ingresso di un'abitazione _GAR_013
-rw-r--r--@   1 mos  staff    798 18 Jul 10:41 APAMEA Museo - Capitello a cestello bizantino _GAR_041
-rw-r--r--@   1 mos  staff    706 18 Jul 10:41 APAMEA Museo - Cortile _GAR_031


## Tests

In [ ]:
depicted_place = "|depicted place = {{city|" + place_mappings_specific.loc[row["Luogo"]]["wikidata"] + "}}"

In [15]:
place_mappings_specific

,Luogo,Nome monumento,category,wikidata
Specific place,,,,
Aleppo Cittadella,Aleppo,Cittadella,Category:Aleppo citadel,d:Q206231
Palmira Tempio di Baal,Palmira,Tempio di Baal,Category:Temple of Baal in Palmyra,d:Q797481
Aleppo Suq,Aleppo,Suq,Category:Aleppo,NaN
Qasr al-Hayr al-Sharqi Palazzo Omayyade,Qasr al-Hayr al-Sharqi,Palazzo Omayyade,NaN,NaN
Yabroud Necropoli di Yabroud,Yabroud,Necropoli di Yabroud,NaN,NaN
Sergiopolis Cattedrale bizantina,Sergiopolis,Cattedrale bizantina,NaN,NaN
Halabiyah Fortezza,Halabiyah,Fortezza,NaN,NaN
Apamea Tra i bambini del villaggio arabo,Apamea,Tra i bambini del villaggio arabo,NaN,NaN
Mari Palazzo di Zimri-Lin,Mari,Palazzo di Zimri-Lin,NaN,NaN
